In [1]:
#import some libraries
import dic
import cv2 as cv
import numpy as np

In [2]:
#load image set from directory
image_set = dic.LoadImages()

#load settings from text file
settings = dic.LoadSettings(image_set)

#no of images in directory
N_images = len(image_set)

In [3]:
#assign reference and deformed image objects
F = dic.ReferenceImage(cv.imread("images/{}".format(image_set[0]),0), settings)
G = dic.DeformedImage(cv.imread("images/{}".format(image_set[1]),0), settings)

In [4]:
#only two images in correlation set
G.sub_centres[0,:] = F.sub_centres[0,:] + np.round(F.P[0,:])
G.sub_centres[1,:] = F.sub_centres[1,:] + np.round(F.P[3,:])

#initial estimate of the shape function parameters as rigid body translation
G.P[0,:], G.P[3,:] = dic.EstimateDisplacementsFourier(F, G)

In [5]:
#no of subsets for correlation run
N_subsets = F.sub_centres.shape[1]

#correlate all subsets
for i in range(0,N_subsets):
    f, f_mean, f_tilde, dfdx, dfdy  = dic.RefSubsetInfo(F,i)
    #
    Hess, fgrad_X_dWdP = dic.Hessian(dfdx, dfdy, F)
    #
    deltaP =  np.ones([6,1])
    itera = 0
    while itera < 100:
        #deform square subset with linear transformation based on current estimation of warp function parameters (u   ux  uy  v   vx  vy)
        deformed_subset = dic.AffineTrans(G,i)
        #extract subset from interpolated G
        g, g_mean, g_tilde = dic.DefSubsetInfo(G, deformed_subset, i)
        stop_val = dic.StopCriteria(deltaP, 0.5*(G.sub_size-1))
        if stop_val < 1e-4:
            #Zero-Mean Normalised Cross Correlation Criteria
            corr_coeff = 1 - sum(((f[:]-f_mean)/f_tilde-(g[:]-g_mean)/g_tilde)**2)/2
            G.stop_val[0,i] = stop_val
            G.iterations[0,i]  = itera
            break 
        else:
            Jacobian = np.dot(fgrad_X_dWdP.T, (f[:]-f_mean-f_tilde/g_tilde*(g[:]-g_mean)))
            deltaP = np.linalg.solve(-Hess, Jacobian) #-1*np.linalg.inv(Hess)*Jacobian.T
            Pupdate = dic.UpdateSFP(G.P[:,i], deltaP)
            G.P[:,i:i+1] = Pupdate
        itera = itera + 1

c:\Users\ed\Documents\GitHub\run-dic\2D_Subset_Based_DIC\dic.py:595: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  w_dP = np.array([


KeyboardInterrupt: 

In [ ]:
#write results to txt csv file

np.savetxt("corr_results6.csv", G.P, 
            delimiter = ",")